# Description

---
.


THIS IS A MONTHLY NOTEBOOK

.

---



This notebook is used to extract ET and other data for analysis in other notebooks/code.

The SEBAL calculations for ET are leveraged from https://github.com/gee-hydro/geeSEBAL

Usage:

  1) Show map with labeled samples for rain or irrigated locations

  2) Manually specify Area of Interest (AOI) with bounding box on map

  3) Specify the filename and number of sample locations to generate within the AOI

  4) Launch batch data extraction


# Setup Notebook

In [1]:
#!pip install geemap
#!pip install geopandas

In [2]:
import subprocess
from tqdm.notebook import tqdm

In [3]:
import ee
ee.Authenticate()
ee.Initialize(project='second-impact-342800')
import geemap

import numpy as np
import pandas as pd
import datetime
from timeit import default_timer as timer
from datetime import timedelta

Enter verification code: 4/1AfJohXnl8V2PYHCUKFNrsuUBpsJujHwKg4f5Csx6xU5bMIkBh7rornfGKgY

Successfully saved authorization token.


In [4]:
# this is used on a cloud server
#%cd /content
#!git clone https://github.com/gee-hydro/geeSEBAL.git
#!git clone https://github.com/earthdataanalytics/irrigation.git
#!cp -r geeSEBAL/etbrasil/ irrigation/
#%cd irrigation/

### Import Custom Libraries

In [5]:
from pipeline import boundaries as bnd
from pipeline import labeledsamples as ls
from pipeline import dataextract as dx
from pipeline import cropmasks as msk

## 1) Show labeled locations on map

Rain = Red = 0

Irrigated = Blue = 1

In [6]:
# labeled MONTHLY data
sample_locations = ls.getRawMonthlyLabeledData()
irr_west = sample_locations.filter(ee.Filter.eq('POINT_TYPE', 1))
rain_west = sample_locations.filter(ee.Filter.eq('POINT_TYPE', 0))

In [7]:
#with ee.profilePrinting():
if True:
    ee.data.setDefaultWorkloadTag('hydracarta_backend')
    ee.data.setWorkloadTag('hydracarta_view_training_labels')

    Map = geemap.Map()
    Map.add_basemap('HYBRID')
    Map.addLayer(irr_west, { 'color': 'blue'}, 'irrigated', opacity=0.5)
    Map.addLayer(rain_west, { 'color': 'red'}, 'rainfed', opacity=0.5)
    Map.center_object(irr_west)

## 2) Draw a bounding box on the map around the area in which to extract data

In [8]:
Map

Map(center=[37.57832872246989, -120.54743003193794], controls=(WidgetControl(options=['position', 'transparent…

## 3) Generate sample locations within the bounding box

Specify the:

* name of the AOI
* number of sample locations to generate


This displays the generated samples as Yellow points on the map.

In [9]:
# specify manually
aoi_name = '20231105_ssebop' # used in the saved filename
num_samples = 100 # number of samples for each class

#with ee.profilePrinting():
if Map and (len(Map.draw_features) > 0):
    AOI = ee.FeatureCollection(Map.draw_features)
    sample_locs = ls.generateMonthlySampleLocations(AOI, aoi_name, 
                                              num_samples=num_samples,
                                              #start_yr=2015, end_yr=2021,
                                              excludeNonCrop=True
                                            )
    rain_count = sample_locs.filter(ee.Filter.eq('POINT_TYPE', 0)).size().getInfo()
    irr_count = sample_locs.filter(ee.Filter.eq('POINT_TYPE', 1)).size().getInfo()
    print(f'Number of sample points generated by type: rain={rain_count}, irr={irr_count}')
    Map.addLayer(sample_locs.draw(color="yellow", pointRadius=5, strokeWidth=3), {}, 'sample_locations', opacity=0.5)
    Map.centerObject(sample_locs)

else:
    print("No AOI selected on Map")

Number of sample points generated by type: rain=100, irr=100


## 4) Launch batch data extraction

In [11]:
if sample_locs:
    #with ee.profilePrinting():
    if True:
        ee.data.resetWorkloadTag()
        ee.data.setWorkloadTag('hydracarta_export_training_labels')
        export_tasks = dx.extractMonthlyData(sample_locs, aoi_name, ET_model=dx.SupportedModels.SSEBOP)#, scale=250)#, restart_index=1825)

Number of samples to extract = 191


Number of tasks launched = 191


In [12]:
!earthengine task list --status RUNNING READY | wc -l

      24


In [ ]:
#!earthengine task cancel all

# Retrieve cost information

In [ ]:
STOP - Wait to continue until all tasks are completed

In [14]:
!earthengine task list -l >> task_list_training.txt

In [15]:
with open('task_list_training.txt') as f:
    task_ids = [x.split(' ')[0] for x in f.readlines()]
#task_ids

In [16]:
all_stats = []

for task_id in tqdm(task_ids):
    task_stats = ee.data.getTaskStatus(task_id)
    all_stats.extend(task_stats)
    
df = pd.DataFrame(all_stats)

  0%|          | 0/191 [00:00<?, ?it/s]

In [17]:
df.to_csv('gee_batch_cost_data_training.csv')

In [19]:
total_batch_eecu_secs = df.batch_eecu_usage_seconds.sum()
cost_per_batch_eecu_hour = 0.4 # $0.4 per eecu-hour
total_cost = total_batch_eecu_secs / 3600 * cost_per_batch_eecu_hour

print(f'Total EECU-secs:   {total_batch_eecu_secs:>10,.0f}')
print(f'Total EECU-hours:  {total_batch_eecu_secs / 3600:>10,.2f}')
print(f'Total cost:       ${total_cost:>10,.2f}')

Total EECU-secs:        3,921
Total EECU-hours:        1.09
Total cost:       $      0.44
